<a href="https://colab.research.google.com/github/c-marq/cap4767-data-mining/blob/main/exercises/week04_group_exercise_session1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 4 Group Exercise — Session 1: Churn EDA + Logistic Regression
**CAP4767 Data Mining with Python** | Miami Dade College — Kendall Campus

**Points:** 10 | **Duration:** ~45 minutes | **Deliverable:** Completed notebook + 2–3 minute presentation

**Objective:** Perform statistical EDA on a churn dataset, estimate the business cost of churn, build a logistic regression baseline, and interpret coefficients as actionable business insights.

**Chapter 4 skills practiced:**
- Cramér's V (categorical association)
- Mann-Whitney U + Cohen's d (continuous feature analysis)
- Business cost estimation
- Logistic regression + coefficient interpretation

### Group Members & Roles

| Role | Name | Responsibility |
|------|------|----------------|
| 🖥️ **Lead Coder** | | Drives the notebook |
| 📊 **Data Interpreter** | | Reads outputs, explains metrics |
| 🎤 **Presenter** | | Delivers the 2–3 minute share-out |
| ✅ **QA Reviewer** | | Checks outputs against checkpoints |

*If 3 members, Lead Coder also handles QA.*

<div style="background-color: #D6EAF8; border-left: 5px solid #2E86C1; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1A5276;">💡 GROUP DISCUSSION (before coding — 3 minutes)</strong><br>
  You're a data team at a telecom company. Before looking at any data, discuss:
  <ol>
    <li>What features do you <em>think</em> predict customer churn? List your top 3 guesses.</li>
    <li>How would you measure the cost of losing a customer? What numbers would you need?</li>
    <li>Why would a business prefer a model that can <em>explain</em> churn drivers over one that just predicts who will leave?</li>
  </ol>
</div>

**Our group's predictions and reasoning (minimum 3 sentences):**

*(Write here)*

---

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Run the setup cell below. It loads the Telco dataset and creates helper functions for your EDA. <strong>Do not modify.</strong>
</div>

In [ ]:
# ============================================================
# Setup — Run this cell. Do not modify.
# ============================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from scipy.stats import chi2_contingency, mannwhitneyu, pointbiserialr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (classification_report, confusion_matrix,
                             accuracy_score, roc_auc_score)

np.random.seed(42)
plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["figure.dpi"] = 100
sns.set_style("whitegrid")

# Helper functions
def cramers_v(x, y):
    """Cramér's V: association between two categorical variables (0–1)."""
    ct = pd.crosstab(x, y)
    chi2 = chi2_contingency(ct)[0]
    n = ct.sum().sum()
    r, k = ct.shape
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))

def cohens_d(group1, group2):
    """Cohen's d: effect size between two groups."""
    n1, n2 = len(group1), len(group2)
    pooled = np.sqrt(((n1-1)*group1.std()**2 + (n2-1)*group2.std()**2) / (n1+n2-2))
    return (group1.mean() - group2.mean()) / pooled if pooled > 0 else 0

# Load and clean
url = "https://raw.githubusercontent.com/c-marq/cap4767-data-mining/refs/heads/main/data/WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(url)
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
df = df.dropna(subset=["TotalCharges"]).drop(columns=["customerID"])

# Feature lists for your EDA
cat_features = [c for c in df.select_dtypes(include="object").columns if c != "Churn"]
num_features = ["tenure", "MonthlyCharges", "TotalCharges"]

print(f"✅ Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"   Churn rate: {(df['Churn']=='Yes').mean():.1%}")
print(f"   Categorical features ({len(cat_features)}): {cat_features}")
print(f"   Numerical features ({len(num_features)}): {num_features}")
print(f"   Helper functions ready: cramers_v(), cohens_d()")

---
## Task 1 — Cramér's V Analysis (2 pts)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Compute Cramér's V between each feature in <code>cat_features</code> and <code>Churn</code>. Create a sorted horizontal bar chart.
</div>

<div style="background-color: #D6EAF8; border-left: 5px solid #2E86C1; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1A5276;">💡 WHY ARE WE DOING THIS?</strong><br>
  In Week 3, we used Pearson correlation for numeric features. Churn is categorical (Yes/No), so we need Cramér's V instead. It works like correlation but for categories: 0 = no association, 1 = perfect association.
</div>

In [ ]:
# Task 1: Cramér's V for all categorical features vs Churn
# Use the pre-built cramers_v() function and cat_features list
# YOUR CODE HERE


**Interpretation (2–3 sentences):** Which 2 features have the strongest association with churn? Were they in your pre-coding predictions?

*(Write here)*

---
## Task 2 — Mann-Whitney U + Cohen's d (2 pts)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  For each feature in <code>num_features</code>, run Mann-Whitney U and compute Cohen's d between churners and non-churners. Display as a table.
</div>

In [ ]:
# Task 2: Mann-Whitney U + Cohen's d
# Split: churn_yes = df[df["Churn"]=="Yes"], churn_no = df[df["Churn"]=="No"]
# For each num_feature: mannwhitneyu() + cohens_d()
# YOUR CODE HERE


**Interpretation (2–3 sentences):** Which numerical feature has the largest effect size? What does the sign of Cohen's d tell you about churners vs stayers?

*(Write here)*

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #922B21;">🛑 CHECKPOINT 1</strong><br>
  <ul>
    <li><strong>Cramér's V:</strong> Contract and InternetService should be the top 2 (both above 0.2)</li>
    <li><strong>Cohen's d:</strong> tenure should show a large negative effect (churners have shorter tenure)</li>
    <li>If Gender is in your top 3 for either test, double-check — gender is typically negligible here</li>
  </ul>
</div>

---
## Task 3 — Business Cost of Churn (2 pts)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Calculate the estimated annual cost of churn. Use these data points:
  <ul>
    <li><code>df["MonthlyCharges"].mean()</code> — average monthly revenue per customer</li>
    <li><code>(df["Churn"]=="Yes").sum()</code> — number of churned customers</li>
    <li>$300 acquisition cost per replacement customer (industry benchmark)</li>
  </ul>
  Compute: (1) annual revenue at risk, (2) replacement cost, (3) total impact.
</div>

<div style="background-color: #D6EAF8; border-left: 5px solid #2E86C1; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1A5276;">💡 WHY ARE WE DOING THIS?</strong><br>
  Models don't convince leadership — dollars do. Before building any model, we establish what churn <em>costs</em>. This number frames every decision from here: "Our model missed 150 churners × $X per customer = $Y in preventable losses."
</div>

In [ ]:
# Task 3: Business cost of churn
# YOUR CODE HERE


---
## Task 4 — Logistic Regression Baseline (2 pts)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  The preprocessing cell below is pre-filled. Run it, then build a <code>LogisticRegression</code> and print the classification report.
</div>

In [ ]:
# Pre-filled preprocessing — Run this cell. Do not modify.
df_model = df.copy()

replace_cols = ["OnlineSecurity", "OnlineBackup", "DeviceProtection",
                "TechSupport", "StreamingTV", "StreamingMovies", "MultipleLines"]
for col in replace_cols:
    df_model[col] = df_model[col].replace({"No internet service": "No", "No phone service": "No"})

binary_cols = ["Partner", "Dependents", "PhoneService", "PaperlessBilling", "Churn"]
for col in binary_cols:
    df_model[col] = df_model[col].map({"Yes": 1, "No": 0})
df_model["gender"] = df_model["gender"].map({"Male": 1, "Female": 0})
for col in replace_cols:
    df_model[col] = df_model[col].map({"Yes": 1, "No": 0})

df_model = pd.get_dummies(df_model, columns=["InternetService", "Contract", "PaymentMethod"],
                           drop_first=True, dtype=int)

X = df_model.drop(columns=["Churn"])
y = df_model["Churn"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
continuous = ["tenure", "MonthlyCharges", "TotalCharges"]
X_train[continuous] = scaler.fit_transform(X_train[continuous])
X_test[continuous] = scaler.transform(X_test[continuous])

feature_names = X_train.columns.tolist()
print(f"✅ Preprocessing complete — {len(feature_names)} features, {X_train.shape[0]:,} train / {X_test.shape[0]:,} test")

In [ ]:
# Task 4: Build logistic regression
# Use max_iter=1000, random_state=42
# Print classification report with target_names=["Stayed", "Churned"]
# YOUR CODE HERE


<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #922B21;">🛑 CHECKPOINT 2</strong><br>
  <ul>
    <li>Accuracy ≈ 80%</li>
    <li>Recall on "Churned" ≈ 50–55% — the model catches about half of actual churners</li>
    <li>AUC ≈ 0.84</li>
  </ul>
</div>

---
## Task 5 — Coefficient Interpretation + Confusion Matrix (2 pts)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  <ol>
    <li>Plot the top 5 positive and top 5 negative coefficients as a horizontal bar chart</li>
    <li>Generate and display the confusion matrix</li>
    <li>Calculate the dollar cost of the False Negatives (churners the model missed)</li>
  </ol>
</div>

In [ ]:
# Task 5a: Coefficient bar chart — top 5 positive, top 5 negative
# YOUR CODE HERE


In [ ]:
# Task 5b: Confusion matrix + dollar cost of false negatives
# Hint: tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
# Cost per missed churner = average MonthlyCharges × 12
# YOUR CODE HERE


**Interpretation (minimum 4 sentences):** Explain the top 3 churn drivers to a non-technical executive. For each one, suggest a specific business action the company could take.

*(Write here)*

---

## Share-Out (2–3 minutes)

Present to the class:
1. **Top 2 churn drivers** from Cramér's V and Cohen's d — did they match your predictions?
2. **Business cost** — what does churn cost this company annually?
3. **Model performance** — how many churners does the model catch? How many slip through?
4. **One actionable recommendation** based on the coefficients

## Troubleshooting

| Problem | Fix |
|---------|-----|
| `cramers_v()` gives error | Make sure you're passing two Series, not strings: `cramers_v(df["Contract"], df["Churn"])` |
| Cohen's d is 0 for all features | Check your group split: `churn_yes = df[df["Churn"]=="Yes"]` — note the string "Yes" not integer 1 |
| Classification report shows all zeros for one class | Model may be predicting all one class — check `max_iter=1000` |
| Confusion matrix shape error | Make sure you ran the preprocessing cell first |

---
<p style="color:#7F8C8D; font-size:0.85em;">
<em>CAP4767 Data Mining with Python | Miami Dade College | Spring 2026</em><br>
Week 4 Group Exercise — Session 1: Churn EDA + Logistic Regression | 10 Points
</p>